In [ ]:
!pip install openai-whisper  groq gtts gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta

# 🎙️ AI-Powered Voice Assistant with Whisper, Groq, and gTTS

This notebook implements a voice-based AI chatbot using the following technologies:

- **Whisper**: Converts speech input into text.
- **Groq API (Llama3-8B)**: Processes the transcribed text and generates a relevant response.
- **gTTS (Google Text-to-Speech)**: Converts the AI-generated response back into speech.

## 🔹 How It Works:
1. The user speaks into the microphone.
2. The Whisper model transcribes the speech into text.
3. The transcribed text is sent to the Llama3-8B model via the Groq API, generating a response.
4. The response is converted into speech using gTTS.
5. The output includes both the text response and an audio file for playback.

This interactive chatbot is implemented using **Gradio**, allowing easy voice-based interactions. 🚀


In [ ]:
import os
os.environ["GROQ_API_KEY"]=""

In [ ]:
import gradio as gr
import whisper
from groq import Groq
from gtts import gTTS

# Initialize Whisper model for transcription
model = whisper.load_model("base")

# Set up Groq API
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


# Function to query the LLM using Groq API  (ye fx input receive krega or llm se trancribe krega)
def get_llm_response(input_text):
    chat_completion = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": input_text,
        }],
        model="llama3-8b-8192",
    )
    return chat_completion.choices[0].message.content


# Function to convert text to speech using gTTS
def text_to_speech(text,output_audio="output_audio.mp3"):
    tts = gTTS(text)
    tts.save(output_audio)
    return output_audio


def chatbot(audio):
  result=model.transcribe(audio)
  user_text=result['text']
  response_text=get_llm_response(user_text)
  output_audio=text_to_speech(response_text)
  return response_text,output_audio


# Create Gradio interface for microphone input
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Audio(type="filepath"),  # Capturing audio from the microphone
    outputs=[gr.Textbox(),gr.Audio(type="filepath")],  # Outputting audio file path
    live=True)

iface.launch()


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 92.8MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ec1c31e55e256bff6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

# Create the chatbot interface
with gr.Blocks() as iface:
    # Title & Description
    gr.Markdown("# 🎙️ AI Voice Chatbot")
    gr.Markdown("Speak into the microphone, and the AI will transcribe, process, and respond with both text and voice.")

    # Input Section
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="🎤 Speak", interactive=True, elem_id="box-style")

    # Output Section
    with gr.Row():
        text_output = gr.Textbox(label="💬 AI Response", interactive=False, elem_id="box-style")

    with gr.Row():
        audio_output = gr.Audio(type="filepath", label="🔊 AI Voice Response", elem_id="box-style")

    # Create a button for interaction
    submit_btn = gr.Button("🚀 Start Chat")
    submit_btn.click(fn=chatbot, inputs=audio_input, outputs=[text_output, audio_output])

    # Footer Section
    gr.Markdown("<hr>")  # Adds a line separator
    gr.Markdown("<p style='text-align: center; font-size: 14px;'>🌟 Developed by <b>Sheema Masood</b> | Built with <b>Gradio</b> 🌟</p>")


iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2f168aabda5699e085.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
